In [1]:
%load_ext autoreload
%autoreload 2

In [62]:
from contextlib import contextmanager


In [63]:
@contextmanager
def transaction(conn):
    """Contextify SQLite transactions. Use with auto-commit mode. E.g.:
    conn = sqlite3.connect("my.db", isolation_level=None)
    Parameters
    ----------
    conn : SQLite connection
        SQLite connection
    Yields
    ------
    None
        Nothing
    """
    # We must issue a "BEGIN" explicitly when running in auto-commit mode.
    conn.execute("BEGIN")
    try:
        # Yield control back to the caller.
        yield
    except:
        conn.rollback()  # Roll back all changes if an exception occurs.
        raise
    else:
        conn.commit()

In [55]:
def store_meta_data(
    cursor,
    zoom_step,
    max_length,
    assembly,
    chrom_names,
    chrom_sizes,
    tile_size,
    max_zoom,
    max_width,
    version,
    header=[],
):
    cursor.execute(
        """
        CREATE TABLE tileset_info
        (
            zoom_step INT,
            max_length INT,
            assembly text,
            chrom_names text,
            chrom_sizes text,
            tile_size REAL,
            max_zoom INT,
            max_width REAL,
            header text,
            version text
        )
        """
    )

    cursor.execute(
        "INSERT INTO tileset_info VALUES (?,?,?,?,?,?,?,?,?,?)",
        (
            zoom_step,
            max_length,
            assembly,
            "\t".join(chrom_names),
            "\t".join(map(str, chrom_sizes)),
            tile_size,
            max_zoom,
            max_width,
            "\t".join(header),
            version,
        ),
    )

    cursor.commit()

    pass

In [41]:
def _bedpe(
    filepath,
    output_file=None,
    assembly=None,
    importance_column="random",
    has_header=False,
    max_per_tile=100,
    tile_size=1024,
    chromosome=None,
    chromsizes_filename=None,
    chr1_col=1,
    from1_col=2,
    to1_col=3,
    chr2_col=4,
    from2_col=5,
    to2_col=6,
    max_zoom=None,
    sqlite_cache_size=500,  # 500 MB
    sqlite_batch_size=100000,
    verbose=0,
):
    BED2DDB_VERSION = 1

    if verbose > 0:
        print(f"BEDPEDB Version {BED2DDB_VERSION}")

    if filepath == "-":
        f = sys.stdin
    elif filepath.endswith(".gz"):
        f = gzip.open(filepath, "rt")
    else:
        f = open(filepath, "r")

    if output_file is None:
        output_file = filepath
        if filepath.endswith(".gz"):
            output_file = os.path.splitext(output_file)[0]
        output_file = os.path.splitext(output_file)[0] + ".bedpedb"

    if op.exists(output_file):
        os.remove(output_file)

    chrom_info, chrom_names, chrom_sizes = cch.load_chromsizes(
        chromsizes_filename, assembly
    )

    def line_to_dict(line):
        parts = line.split()
        d = {}
        try:
            chrom1 = parts[chr1_col - 1]
            chrom2 = parts[chr2_col - 1]
            chrom1_offset = chrom_info.cum_chrom_lengths[chrom1]
            chrom2_offset = chrom_info.cum_chrom_lengths[chrom2]

            d["xs"] = [
                chrom1_offset + int(parts[from1_col - 1]),
                chrom1_offset + int(parts[to1_col - 1]),
            ]
            d["ys"] = [
                chrom2_offset + int(parts[from2_col - 1]),
                chrom2_offset + int(parts[to2_col - 1]),
            ]
        except KeyError:
            error_str = (
                "ERROR converting chromosome position to genome position. "
                "Please make sure you've specified the correct assembly "
                "using the --assembly option or a chromsizes file using the . "
                "--chromsizes-filename option."
                "Current assembly: {}, chromosomes: {},{}".format(
                    assembly, parts[chr1_col - 1], parts[chr2_col - 1]
                )
            )
            raise (KeyError(error_str))

        d["uid"] = slugid.nice()

        d["chrOffset"] = d["xs"][0] - int(parts[from1_col - 1])
        d["chrom1"] = str(chrom1)
        d["chrom2"] = str(chrom2)

        if importance_column is None:
            d["importance"] = max(d["xs"][1] - d["xs"][0], d["ys"][1] - d["ys"][0])
        elif importance_column == "random":
            d["importance"] = random.random()
        else:
            # We seem to use one-based numbering for columns...
            d["importance"] = float(parts[int(importance_column) - 1])

        d["fields"] = line

        return d

    entries = []

    if has_header:
        f.readline()
    else:
        first_line = f.readline().strip()
        try:
            parts = first_line.split()

            int(parts[from1_col - 1])
            int(parts[to1_col - 1])
            int(parts[from2_col - 1])
            int(parts[to2_col - 1])
        except ValueError:
            error_str = (
                "Couldn't convert one of the bedpe coordinates to an "
                "integer. If the input file contains a header, make sure to "
                "indicate that with the --has-header option. Line: {}".format(
                    first_line
                )
            )
            raise ValueError(error_str)
        entries = [line_to_dict(first_line)]

    entries += [line_to_dict(line) for line in [line.strip() for line in f] if line]

    if chromosome is not None:
        entries = [
            d for d in entries if d["chrom1"] == chromosome or d["chrom2"] == chromosome
        ]

    if verbose > 0:
        print(f"Found {len(entries)} entries")

    # We need chromosome information as well as the assembly size to properly
    # tile this data
    assembly_size = chrom_info.total_length + 1
    max_zoom = int(math.ceil(math.log(assembly_size / tile_size) / math.log(2)))

    # this script stores data in a sqlite database
    sqlite3.register_adapter(np.int64, lambda val: int(val))
    conn = sqlite3.connect(output_file, isolation_level=None)

    # store some meta data
    store_meta_data(
        conn,
        1,
        max_length=assembly_size,
        assembly=assembly,
        chrom_names=chrom_names,
        chrom_sizes=chrom_sizes,
        tile_size=tile_size,
        max_zoom=max_zoom,
        max_width=tile_size * 2 ** max_zoom,
        version=BED2DDB_VERSION,
    )

    # max_width = tile_size * 2 ** max_zoom
    # uid_to_entry = {}

    c = conn.cursor()
    c.execute("PRAGMA synchronous = OFF;")
    c.execute("PRAGMA journal_mode = OFF;")
    c.execute(f"PRAGMA cache_size = {int(sqlite_cache_size * 1000)};")

    c.execute(
        """
        CREATE TABLE intervals
        (
            id int PRIMARY KEY,
            zoomLevel int,
            importance real,
            fromX int,
            toX int,
            fromY int,
            toY int,
            chrOffset int,
            uid text,
            fields text
        )
        """
    )

    c.execute(
        """
        CREATE VIRTUAL TABLE position_index USING rtree(
            id,
            rFromX, rToX,
            rFromY, rToY
        )
        """
    )

    curr_zoom = 0
    counter = 0

    tile_counts = col.defaultdict(lambda: col.defaultdict(lambda: col.defaultdict(int)))
    # Sort from high to low importance
    entries = sorted(entries, key=lambda x: -x["importance"])

    interval_inserts = []
    position_index_inserts = []

    def batch_insert(conn, c, interval_inserts, position_index_inserts):
        if verbose > 0:
            print(f"Insert batch ({counter})")

        with transaction(conn):
            c.executemany(
                "INSERT INTO intervals VALUES (?,?,?,?,?,?,?,?,?,?)", interval_inserts
            )
            c.executemany(
                "INSERT INTO position_index VALUES (?,?,?,?,?)", position_index_inserts
            )

        interval_inserts.clear()
        position_index_inserts.clear()

    for entry_num, d in enumerate(entries):
        curr_zoom = 0

        while curr_zoom <= max_zoom:
            tile_width = tile_size * 2 ** (max_zoom - curr_zoom)
            tile_from = list(
                map(lambda x: int(x / tile_width), [d["xs"][0], d["ys"][0]])
            )
            tile_to = list(map(lambda x: int(x / tile_width), [d["xs"][1], d["ys"][1]]))

            empty_tiles = True

            # go through and check if any of the tiles at this zoom level are
            # full

            for i in range(tile_from[0], tile_to[0] + 1):
                if not empty_tiles:
                    break

                for j in range(tile_from[1], tile_to[1] + 1):
                    if tile_counts[curr_zoom][i][j] > max_per_tile:

                        empty_tiles = False
                        break

            if empty_tiles:
                # they're all empty so add this interval to this zoom level
                for i in range(tile_from[0], tile_to[0] + 1):
                    for j in range(tile_from[1], tile_to[1] + 1):
                        tile_counts[curr_zoom][i][j] += 1

                interval_inserts.append(
                    (
                        counter,
                        curr_zoom,
                        d["importance"],
                        d["xs"][0],
                        d["xs"][1],
                        d["ys"][0],
                        d["ys"][1],
                        d["chrOffset"],
                        d["uid"],
                        d["fields"],
                    )
                )

                position_index_inserts.append(
                    (counter, d["xs"][0], d["xs"][1], d["ys"][0], d["ys"][1])
                )

                counter += 1
                break

            curr_zoom += 1

        if len(interval_inserts) >= sqlite_batch_size:
            batch_insert(conn, c, interval_inserts, position_index_inserts)

    batch_insert(conn, c, interval_inserts, position_index_inserts)

    c.close()

    return

In [5]:
conda create --name=test python=3.10 jupyter pandas ipykernel cytoolz


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [2]:
%%bash
# Create the test environment for this notebook.
# All of these steps will be needed to run it fully.
# Otherwise, you may run separate installations if you need them.

conda create --name=test python=3.10 jupyter pandas ipykernel cytoolz
conda activate test
python -m ipykernel install --user --name=test

# Data loading tools:
## pip install resgen-python
pip install clodius
pip install git+https://github.com/manzt/hg.git@main

# Work with cool files:
pip install cooler bioframe

# Set up widgets for interactivity (might break and always require tinkering):
conda install -n base -c conda-forge jupyterlab_widgets
conda install -n test -c conda-forge ipywidgets
conda install -c conda-forge "nodejs>=12.0.0"
jupyter labextension install @jupyter-widgets/jupyterlab-manager

# Beautiful and formatted visualization of printed output:
conda install -c conda-forge rich

Process is terminated.


# Load data to local higlass instance

In [2]:
import glob
import pandas as pd
import cooler

In [3]:
COOLERS = glob.glob("../arcuda/HiGlass/coolers/*.mapq_30.100.mcool")
CONDITIONS = [x.split('/')[-1].split('.')[0] for x in COOLERS]

### Preliminary work

Make sure to install jupyter, [higlass-python](https://github.com/higlass/higlass-python#installation) and [jupyter widgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html#installation). Widgets are always painful to run, working with them requires lots of googling.

In [4]:
# ! pip install git+https://github.com/manzt/hg.git@main # higlass-python v2

In [5]:
import hg

In [6]:
from rich import print # Useful but not necessary

#### Test on remote mouse datasets

Create `Tilesets` (speacial object for loading various higlass tracks) with hg: 

In [7]:
gene_anno_tileset = hg.remote(
    uid="QDutvmyiSrec5nX4pA5WGQ",
    server="https://higlass.io/api/v1/",
    name="Gene Annotations (mm10)", # optional
)

cooler_tileset = hg.remote(
    uid="eApFIhe2QMOOSV2XO7UT6g",
    server="https://resgen.io/api/v1/gt/paper-data",
    name="Hsieh et al. 2019 MicroC mESC_JM8.N4 - WT", # optional
)

print(gene_anno_tileset)

RemoteTileset(
    uid='QDutvmyiSrec5nX4pA5WGQ',
    server='https://higlass.io/api/v1/',
    name='Gene Annotations (mm10)'
)

From the `Tileset` instance, we can create tracks easily from public datasets:

In [9]:
view = hg.view(
    (gene_anno_tileset.track("horizontal-gene-annotations", height=60), "top"),
    (gene_anno_tileset.track("vertical-gene-annotations", height=60), "left"),
    (cooler_tileset.track("heatmap"), "center"),
)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=[EnumTrack(tilesetUid='QDutvmyiSrec5nX4pA5WGQ', server='https://higlass.io/api/v1/', type='vertical-gene-annotations', uid='b4b7a0a1-b143-46b4-858f-a0796a8fc1f5', width=None, height=60, options={'name': 'Gene Annotations (mm10)'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], right=None, top=[EnumTrack(tilesetUid='QDutvmyiSrec5nX4pA5WGQ', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='f41b99d1-065d-460b-afe9-856a3917e0d5', width=None, height=60, options={'name': 'Gene Annotations (mm10)'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=[HeatmapTrack(tilesetUid='eApFIhe2QMOOSV2XO7UT6g', server='https://resgen.io/api/v1/gt/paper-data', type='heatmap', uid='014ab03a-d879-41e1-b5b3-9af78d1be421', width=None, height=None, options={'name': 'Hsieh et al. 2019 MicroC mESC_JM8.N4 - WT'}, data=None, position=None, transforms=None)], whole=None, gallery=None), uid='fe153cf4-ced3-432c-a838-def3d56fff43', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=None, initialYDomain=None, overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1, None))

#### Test on remotely uploaded resgen datasets:
We will add dots:

In [11]:
gene_anno_tileset = hg.remote(
    uid="QDutvmyiSrec5nX4pA5WGQ",
    server="https://higlass.io/api/v1/",
    name="Gene Annotations (mm10)", # optional
)

cooler_tileset = hg.remote(
    uid="fpI5li2CT1yQXVyN2NvHYA",
    server="https://resgen.io/api/v1",
    name="120min_with_auxin.mm10.mapq_30.100.mcool", # optional
)

# Might be not needed for Schizo:
dots_tileset = hg.remote(
    uid="ODCJkQ4dRkWpCQLPTBBthg",
    server="https://resgen.io/api/v1",
    name="zhang2021NatComm.dots.all.withoffset.bedpe.multires", # optional
)


In [12]:
view = hg.view(
    (gene_anno_tileset.track("horizontal-gene-annotations", height=60), "top"),
    (gene_anno_tileset.track("vertical-gene-annotations", height=60), "left"),
    (cooler_tileset.track("heatmap"), "center"),
    (dots_tileset.track("2d-rectangle-domains"), "center"),
)
view

View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=[EnumTrack(tilesetUid='QDutvmyiSrec5nX4pA5WGQ', server='https://higlass.io/api/v1/', type='vertical-gene-annotations', uid='3d667d78-de68-44b3-9d4d-b4d547c33835', width=None, height=60, options={'name': 'Gene Annotations (mm10)'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], right=None, top=[EnumTrack(tilesetUid='QDutvmyiSrec5nX4pA5WGQ', server='https://higlass.io/api/v1/', type='horizontal-gene-annotations', uid='4dd1424a-4770-4053-9db5-dc742a5bd444', width=None, height=60, options={'name': 'Gene Annotations (mm10)'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=[HeatmapTrack(tilesetUid='fpI5li2CT1yQXVyN2NvHYA', server='https://resgen.io/api/v1', type='heatmap', uid='89c95df7-5664-4a39-b635-465b74947728', width=None, height=None, options={'name': '120min_with_auxin.mm10.mapq_30.100.mcool'}, data=None, position=None, transforms=None), EnumTrack(tilesetUid='ODCJkQ4dRkWpCQLPTBBthg', server='https://resgen.io/api/v1', type='2d-rectangle-domains', uid='48482e14-cc75-46d6-9bd6-93a7782130a8', width=None, height=None, options={'name': 'zhang2021NatComm.dots.all.withoffset.bedpe.multires'}, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], whole=None, gallery=None), uid='4b91b666-da4f-4171-9e05-6de56a430795', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=None, initialYDomain=None, overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1, None))

#### Local Tilesets

Now let's try to load th
An important feature of `hg` is its ability to host local data-sources (via a background server) to power a HiGlass visualization. Currently only `bigwig`, `cooler` and `multivec` formats are implemented, but _any_ implementation from `clodius.tiles` can easily be supported (https://github.com/higlass/clodius/tree/develop/clodius/tiles). 

In [12]:
# !pip install clodius # using the local server requires clodius to be installed

In [13]:
import hg

In [14]:
hg.server.enable_proxy()

In [15]:
import os
import glob

In [22]:
COOLERS = glob.glob("../arcuda/HiGlass/coolers/*.mapq_30.1000.mcool")
CONDITIONS = [x.split('/')[-1].split('.')[0] for x in COOLERS]

In [23]:
COOLERS

['../arcuda/HiGlass/coolers/WT.danrer11-reduced.mapq_30.1000.mcool']

In [99]:
import os.path as op
import clodius.chromosomes as cch
import slugid
import math
import sqlite3
import numpy as np
import collections as col

In [100]:
_bedpe('../arcuda/HiGlass/boundaries/tads.bed',  
       importance_column=4, 
       chromsizes_filename='../arcuda/genome/danRer11.reduced.chromsizes', 
       chr1_col=1, chr2_col=7, 
       from1_col=2, from2_col=3, 
       to1_col=5, to2_col=6)

FileNotFoundError: [Errno 2] No such file or directory: '../arcuda/HiGlass/boundaries/tads.bed'

In [110]:
# Might require two times to run
cooler_tileset = hg.cooler(sorted(COOLERS)[0]) # returns a `LocalTileset` which can be used identially to a `RemoteTileset`
tads = hg.bed2ddb('../arcuda/HiGlass/boundaries/tads/tads.bedpedb')

boundaries_all = hg.bigwig('../arcuda/HiGlass/boundaries/bw/boundaries_WT_all_clusters.bw')
boundaries_0 = hg.bigwig('../arcuda/HiGlass/boundaries/bw/boundaries_WT_cluster_0.bw')
boundaries_1 = hg.bigwig('../arcuda/HiGlass/boundaries/bw/boundaries_WT_cluster_1.bw')
boundaries_2 = hg.bigwig('../arcuda/HiGlass/boundaries/bw/boundaries_WT_cluster_2.bw')
boundaries_3 = hg.bigwig('../arcuda/HiGlass/boundaries/bw/boundaries_WT_cluster_3.bw')

fountains_all = hg.bigwig('../arcuda/HiGlass/fountains/bw/fountains_WT_all_clusters.bw')
fountains_0 = hg.bigwig('../arcuda/HiGlass/fountains/bw/fountains_WT_cluster_0.bw')
fountains_1 = hg.bigwig('../arcuda/HiGlass/fountains/bw/fountains_WT_cluster_1.bw')
fountains_2 = hg.bigwig('../arcuda/HiGlass/fountains/bw/fountains_WT_cluster_2.bw')
fountains_3 = hg.bigwig('../arcuda/HiGlass/fountains/bw/fountains_WT_cluster_3.bw')
fountains_4 = hg.bigwig('../arcuda/HiGlass/fountains/bw/fountains_WT_cluster_4.bw')

view = hg.view(
    (cooler_tileset.track("horizontal-chromosome-labels"), "top"),
    #(gene_anno_tileset.track("horizontal-gene-annotations", height=60), "top"),
    (cooler_tileset.track("vertical-chromosome-labels"), "left"),
    (cooler_tileset.track("heatmap"), "center"),
    #(tads.track("2d-rectangle-domains"), "center"),
    (boundaries_all.track("bar"), "top"),
    #(boundaries_0.track("bar"), "top"),
    #(boundaries_1.track("bar"), "top"),
    #(boundaries_2.track("bar"), "top"),
    #(boundaries_3.track("bar"), "top"),
    (fountains_all.track("bar"), "top"),
    #(fountains_0.track("bar"), "top"),
    #(fountains_1.track("bar"), "top"),
    #(fountains_2.track("bar"), "top"),
    #(fountains_3.track("bar"), "top"),
    #(fountains_4.track("bar"), "top"),
)

view

# Lock zoom & location for each `View`
# view_lock = hg.lock(view1, view2)

# Concatenate views horizontally and apply synchronization lock
# (view1 | view2).locks(view_lock)



View(layout=Layout(x=0, y=0, w=12, h=6, moved=None, static=None), tracks=Tracks[TrackT](left=[EnumTrack(tilesetUid='1570b703fb82b7bc68a8a133002e460a', server='/proxy/50396/api/v1/', type='vertical-chromosome-labels', uid='a864b977-e890-421f-8557-bd6198d74b36', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], right=None, top=[EnumTrack(tilesetUid='1570b703fb82b7bc68a8a133002e460a', server='/proxy/50396/api/v1/', type='horizontal-chromosome-labels', uid='8e8a511f-eb74-45a8-9234-9115cb4516d5', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='f97406122ec4649888e00dc137a9f234', server='/proxy/50396/api/v1/', type='bar', uid='7fa81d75-1aef-4b59-9aab-9a6ba3aaf2a2', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None), EnumTrack(tilesetUid='6bb03a499236059bd028775105f27c5f', server='/proxy/50396/api/v1/', type='bar', uid='92171668-cc8d-41ba-b28b-f8d2dac1936f', width=None, height=None, options=None, data=None, chromInfoPath=None, fromViewUid=None, x=None, y=None)], bottom=None, center=[HeatmapTrack(tilesetUid='1570b703fb82b7bc68a8a133002e460a', server='/proxy/50396/api/v1/', type='heatmap', uid='aba1922f-ddda-4d1f-92cc-74f4a187bfa3', width=None, height=None, options=None, data=None, position=None, transforms=None)], whole=None, gallery=None), uid='bc71b1ae-3fda-442d-8a01-87785e17715b', autocompleteSource=None, chromInfoPath=None, genomePositionSearchBox=None, genomePositionSearchBoxVisible=None, initialXDomain=None, initialYDomain=None, overlays=None, selectionView=None, zoomFixed=None, zoomLimits=(1, None))

## Visual inspection of the clusters

In [90]:
df_dots = pd.read_csv("../arcuda/HiGlass/boundaries/boundaries_WT_cluster_3.bed", sep='\t', header=None)

In [91]:
df_dots.columns = ['chrom1', 'start1', 'end1', 'value1']

In [96]:
import bioframe

chromsizes = cooler_tileset.tileset.info()["chromsizes"]

# Sort dots by their position on chromosome:
df_dots = bioframe.sort_bedframe(df_dots, bioframe.from_dict(chromsizes), cols=('chrom1', 'start1', 'end1'))

# Create list of dots starts:
dots_positions = [f"{x[0]} {x[1]}" for x in df_dots[['chrom1', 'start1']].drop_duplicates().values]
offsets = [x[1] for x in df_dots[['chrom1', 'start1']].drop_duplicates().values]

In [97]:
## Example of zooming into chromosomes:
import itertools
from operator import itemgetter

# # extract chromsizes from tileset and compute absolute offsets
chromsizes = cooler_tileset.tileset.info()["chromsizes"]
offsets = list(itertools.accumulate(map(itemgetter(1), chromsizes), initial=0))
chromsizes, offsets

([['chr1', 59578282],
  ['chr2', 59640629],
  ['chr3', 62628489],
  ['chr4', 78093715],
  ['chr5', 72500376],
  ['chr6', 60270059],
  ['chr7', 74282399],
  ['chr8', 54304671],
  ['chr9', 56459846],
  ['chr10', 45420867],
  ['chr11', 45484837],
  ['chr12', 49182954],
  ['chr13', 52186027],
  ['chr14', 52660232],
  ['chr15', 48040578],
  ['chr16', 55266484],
  ['chr17', 53461100],
  ['chr18', 51023478],
  ['chr19', 48449771],
  ['chr20', 55201332],
  ['chr21', 45934066],
  ['chr22', 39133080],
  ['chr23', 46223584],
  ['chr24', 42172926],
  ['chr25', 37502051],
  ['chrM', 16596]],
 [0,
  59578282,
  119218911,
  181847400,
  259941115,
  332441491,
  392711550,
  466993949,
  521298620,
  577758466,
  623179333,
  668664170,
  717847124,
  770033151,
  822693383,
  870733961,
  926000445,
  979461545,
  1030485023,
  1078934794,
  1134136126,
  1180070192,
  1219203272,
  1265426856,
  1307599782,
  1345101833,
  1345118429])

In [98]:
import ipywidgets

# a jupyter widget for the higlass viewer
higlass_widget = view.widget()

# a dropdown widget
dropdown = ipywidgets.Dropdown(options=dots_positions)

def handle_change(e):
    """Connects dropdown menu to HiGlassWidget instance"""
    
    # make sure it's a "change" event and the value of "new" is the index of the option
    if e["type"] == "change" and e["name"] == "index":
        index = e["new"]
        
        start = offsets[index] - 100_000
        end = offsets[index] + 100_000
        
        # navigate higlass viewer
        higlass_widget.zoom_to(
            view.uid,
            start, end,
            start, end)
    

# connect our function above to the dropdown menu
dropdown.observe(handle_change)

# NB - there seems to be a bug the first time you engage with the dropdown. Try again and things should work...
ipywidgets.VBox([dropdown, higlass_widget])

IndexError: list index out of range